In [1]:
# get ratios for segment analysis 
# use BL of whole campaign 
# date created: 3/2/2020
# author: sofia chelpon 


In [2]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns

import datetime as dt 

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [4]:
#### ------------------- LOAD IN 
# data overall 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df.pkl")

# segment data
awas_segments = pd.read_pickle("./awas_segments.pkl")
awas_segments = awas_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON').drop(columns = 'Notes')
# segment data 
toga_segments = pd.read_pickle("./toga_segments.pkl")
toga_segments = toga_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON')

In [5]:
##### --------------------- get campaign average BL means 
# AWAS 
awas_bl = pd.Series()
awas_bl = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()
awas_bl = awas_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
awas_bl.index.name = "Trace_Gas"

# TOGA
toga_bl = pd.Series()
toga_bl = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()
toga_bl = toga_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
toga_bl.index.name = "Trace_Gas"

In [7]:
######## ------------- GET AWAS RATIOS 
# thing to fill with ratios 
awas_seg_ratios = pd.DataFrame()
for col in awas_df.columns: 
    awas_seg_ratios[col] = ''
awas_seg_ratios = awas_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(awas_segments))):
    # find segment, get info 
    awas_ut = awas_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': awas_ut['Time_UTC'], 'Flight': awas_ut['Flight']})
    # get ratio 
    awas_ut = awas_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = awas_ut/awas_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    awas_seg_ratios = awas_seg_ratios.append(to_append, ignore_index = True)

In [8]:
######## ------------- GET TOGA RATIOS 
# thing to fill with ratios 
toga_seg_ratios = pd.DataFrame()
for col in toga_df.columns: 
    toga_seg_ratios[col] = ''
toga_seg_ratios = toga_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(toga_segments))):
    # find segment, get info 
    toga_ut = toga_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': toga_ut['Time_UTC'], 'Flight': toga_ut['Flight']})
    # get ratio 
    toga_ut = toga_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = toga_ut/toga_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    toga_seg_ratios = toga_seg_ratios.append(to_append, ignore_index = True)


In [9]:
### add attribute to each species (BL, tropo, and UT lifetimes)
fill = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], columns=['Trace_Gas', 'BL_tau', 'TROPO_tau', 'UT_tau'])
fill = fill.append(fill)

# --------- AWAS 
# read in from xls file to make dataframe - awas
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_12162019.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
awas_lifetimes = fill.append(awas_lifetimes)
awas_lifetimes = awas_lifetimes.reset_index().drop('index', axis = 1)

# --------- TOGA
# read in from xls file to make dataframe - toga 
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_12162019.xlsx'
toga_lifetimes = pd.read_excel(mypath2)
toga_lifetimes = fill.append(toga_lifetimes)
toga_lifetimes = toga_lifetimes.reset_index().drop('index', axis = 1)

In [10]:
# combine data from both instruments to one dataframe 
# --------- AWAS 
awas_seg_ratios = awas_seg_ratios.transpose()
awas_seg_ratios.insert(0, 'Instrument', 'AWAS')

# add lifetimes 
awas_seg_ratios.insert(1, 'BL_tau', awas_lifetimes['BL_tau'].values)
awas_seg_ratios.insert(2, 'TROPO_tau', awas_lifetimes['TROPO_tau'].values)
awas_seg_ratios.insert(3, 'UT_tau', awas_lifetimes['UT_tau'].values)

# --------- TOGA 
toga_seg_ratios = toga_seg_ratios.transpose()
toga_seg_ratios.insert(0, 'Instrument', 'TOGA')

# add lifetimes 
toga_seg_ratios.insert(1, 'BL_tau', toga_lifetimes['BL_tau'].values)
toga_seg_ratios.insert(2, 'TROPO_tau', toga_lifetimes['TROPO_tau'].values)
toga_seg_ratios.insert(3, 'UT_tau', toga_lifetimes['UT_tau'].values)

In [12]:
# combine data from both instruments to one dataframe 
toga_seg_ratios = toga_seg_ratios.drop('Time_UTC', axis = 0)
toga_seg_ratios = toga_seg_ratios.drop('Flight', axis = 0)

ratios_comb = awas_seg_ratios.append(toga_seg_ratios)
ratios_comb

,Instrument,BL_tau,TROPO_tau,UT_tau,0,1,2,3,4,5,...,196,197,198,199,200,201,202,203,204,205
Time_UTC,AWAS,NaN,NaN,NaN,2014-01-22 01:44:00,2014-01-22 01:55:15,2014-01-22 02:06:30,2014-01-22 02:17:45,2014-01-22 02:29:00,2014-01-22 02:37:56,...,2014-02-25 02:39:55,2014-02-25 02:45:55,2014-02-25 02:54:56,2014-02-25 03:03:56,2014-02-25 03:12:56,2014-02-25 03:18:20,2014-02-25 03:27:20,2014-02-25 03:36:20,2014-02-25 03:49:22,2014-02-25 05:19:51
Flight,AWAS,NaN,NaN,NaN,RF05,RF05,RF05,RF05,RF05,RF05,...,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14
C2H6_Ethane,AWAS,18.555721,48.949092,167.260709,0.50874,0.513608,0.438149,0.418676,0.676697,0.481964,...,0.138747,0.159438,0.132662,0.163089,0.153352,0.0973665,0.324961,0.317658,0.443018,0.612192
C2H2_Ethyne,AWAS,5.422335,9.603369,18.712675,0.313679,0.346698,0.28066,0.28066,0.619104,0.321934,...,0.0660377,0.0825472,0.057783,0.0825472,0.0742925,0.0495283,0.15684,0.14033,0.346698,0.396226
C3H8_Propane,AWAS,4.214557,9.062693,22.450000,0.153034,0.173903,0.104342,0.104342,0.612137,0.194771,...,0.0347805,0.0208683,0.0347805,0.0347805,0.0417366,0.0417366,0.0556488,0.0556488,0.139122,0.236508
C4H10_Isobutane,AWAS,2.175570,4.031311,7.903592,0.0730395,0.0929594,0.0597596,0.0597596,0.683915,0.139439,...,0.0331998,0.0863194,0.0331998,0.0464797,0.0464797,0.0331998,0.0398397,0.0531196,0.0531196,0.119519
C4H10_nButane,AWAS,1.959376,3.813463,8.057685,0.0622095,0.0912406,0.103682,0.0787987,0.67601,0.15345,...,0.0787987,0.0497676,0.0539149,0.0663568,0.041473,0.0497676,0.0539149,0.0539149,0.0539149,0.132714
C5H12_nPentane,AWAS,1.214449,2.312606,4.723079,NaN,NaN,0.0478984,NaN,0.502933,0.0239492,...,NaN,NaN,NaN,0.0478984,0.0957967,NaN,NaN,NaN,NaN,NaN
C6H6_Benzene,AWAS,3.755136,6.447323,11.947207,0.171265,0.182435,0.134034,0.13031,0.584536,0.193604,...,0.0409547,0.0372316,0.0335084,0.0372316,0.0335084,0.0335084,0.0372316,0.0335084,0.07074,0.27179
OCS_Carbonyl_sulfide,AWAS,2000.000000,2000.000000,2000.000000,1.00398,0.989348,1.00398,1.02409,1.02592,1.01495,...,0.835734,0.857679,0.828419,0.839392,0.817447,0.762585,1.02958,0.866823,0.888768,1.03141


In [13]:
# pickle this!! 
ratios_comb.to_pickle("./contrast_ratios_ut_seg.pkl")